In [43]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [44]:
data = pd.read_csv('../../../../dataset/id-multi-label-hate-speech-and-abusive-language-detection/re_dataset.csv', encoding='latin-1')
alay_dict = pd.read_csv('../../../../dataset/id-multi-label-hate-speech-and-abusive-language-detection/new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})

In [45]:
def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('url', ' ', text) # Remove every URL
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub(r'\b(?:x[a-fA-F0-9]{2}\s*)+\b', '', text) # Remove emoji bytecode
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def preprocess(text):
    text = lowercase(text)
    text = remove_nonaplhanumeric(text)
    text = remove_unnecessary_char(text)
    text = normalize_alay(text) 

    return text

In [46]:
data['Tweet'] = data['Tweet'].apply(preprocess)

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=False)
train_data.sort_values(by=['Tweet'], inplace=True)

train_labels = train_data.columns[1:]
val_labels = val_data.columns[1:]

# Extract features and labels for training and validation
X_train = train_data['Tweet'].values
y_train = train_data[train_labels].values
X_val = val_data['Tweet'].values
y_val = val_data[val_labels].values

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

print(X_train[-3:])

acquired_data = pd.read_csv('../acquired_data/hsd-coreset-1-data-10535.csv')
acquired_data.sort_values(by=['processed_text'], inplace=True)

acq_X_train = acquired_data['processed_text'].values
acq_y_train = acquired_data[acquired_data.columns[1:]].values

print(acq_X_train.shape, acq_y_train.shape)
print(acq_X_train[-3:])

(10535,) (10535, 12)
(2634,) (2634, 12)
['yuk datang dalam acara launching kajian ekonomi syariat di di jaka a pada hari jumat 20 april 2018 pukul 12 45 14 00 ba dan jumatan tema berinvestasi di pasar modal syariat halal atau haram narasumber ustad mahbub sama afi ram'
 'yusril halang halang i pembubaran hati mau lengserkan jokowi '
 'yusril kelompok islam te indas di era jokowi fakta 2019 ganti presiden']
(10535,) (10535, 12)
['yuk datang dalam acara launching kajian ekonomi syariat di di jaka a pada hari jumat 20 april 2018 pukul 12 45 14 00 ba dan jumatan tema berinvestasi di pasar modal syariat halal atau haram narasumber ustad mahbub sama afi ram'
 'yusril halang halang i pembubaran hati mau lengserkan jokowi '
 'yusril kelompok islam te indas di era jokowi fakta 2019 ganti presiden']


In [47]:
def verify(ori_x, acquired_x):
    ori_x_sorted = np.sort(ori_x)
    acquired_x_sorted = np.sort(acquired_x)

    if np.array_equal(ori_x_sorted, acquired_x_sorted):
        print("ori_x and acquired_x contain the same elements (ignoring order).")
    else:
        print("ori_x and acquired_x have different elements.")

    set_ori_x = set(ori_x)
    set_acquired_x = set(acquired_x)

    print("Elements in ori_x but not in acquired_x:", len(set_ori_x - set_acquired_x))
    print("Elements in acquired_x but not in ori_x:", len(set_acquired_x - set_ori_x))

    diff_indices = np.where(ori_x != acquired_x)[0]  # Get indices where elements differ
    print("Mismatched indices:", diff_indices)
    print("ori_x mismatches:", ori_x[diff_indices])

In [48]:
verify(X_train, acq_X_train)

ori_x and acquired_x contain the same elements (ignoring order).
Elements in ori_x but not in acquired_x: 0
Elements in acquired_x but not in ori_x: 0
Mismatched indices: []
ori_x mismatches: []
